In [1]:
import networkx as nx
from dataclasses import dataclass
from itertools import repeat
from matplotlib import pyplot
from operator import itemgetter
from scipy.optimize import linprog
from typing import Any
from typing import ClassVar
from typing import Dict
from typing import List
from typing import NoReturn
from typing import Optional
from typing import Set
from typing import Tuple


@dataclass
class Coord:
    x:          int           = 0
    y:          int           = 0
    COORD_ATTR: ClassVar[str] = "coord"
        
    def copy(self, x: int = None, y: int = None):
        return Coord(x = x or self.x, y = y or self.y)

In [2]:
class AcyclicGraphDrawer:
    def __init__(self, path: str, layer_wide: Optional[int] = None):
        self.layer_wide: Optional[int]              = layer_wide
        self.graph:      nx.classes.digraph.DiGraph = nx.read_graphml(path)
        self.nodes:      List[str]                  = list(self.graph.nodes)
        self.id_by_node: Dict[str, int]             = {node: node_id 
                                                       for node_id, node in enumerate(self.nodes)
                                                      }
        self.nodes_num:  int                        = len(self.nodes)
        self.edges:      List[Tuple[str, str]]      = self.graph.edges.keys()
        self.rev_edges:  List[Tuple[str, str]]      = self.graph.reverse(copy=True).edges.keys()
        self.pi:         Dict[str, float]           = dict(zip(self.nodes, repeat(float("inf"))))
        self.pic_path:   str                        = path.replace("xml", "png")
        
    def __set_layer_by_node(self, layers: List[Set[str]]) -> NoReturn:
        self.layer_by_node: Dict[str, int] = {node: layer_id
            for layer_id, layer in enumerate(layers) for node in layer
        }
        
    def __solver(self) -> Dict[str, Any]:
        c:    List[int]       = [0 for _ in range(self.nodes_num)]
        A_ub: List[List[int]] = []
        
        for from_node, to_node in self.edges:
            row: List[int] = [0 for _ in range(self.nodes_num)]
            row[self.id_by_node[from_node]] = -1
            row[self.id_by_node[to_node]] = 1
            c[self.id_by_node[from_node]] += 1
            c[self.id_by_node[to_node]] -= 1
            A_ub.append(row)

        return linprog(c=c, 
                       A_ub=A_ub, 
                       b_ub=[-1 for _ in range(len(A_ub))], 
                       bounds=[(0, None) for _ in range(self.nodes_num)],
                       method='revised simplex')
        
    def compute_layers(self) -> List[Set[str]]:
        node_layers:   List[float]    = self.__solver()['x']
        max_layer_num: int            = int(max(node_layers))
        layers:        List[Set[str]] = [set() for i in range(max_layer_num + 1)]

        for layer_id, layer in enumerate(node_layers):
            layers[int(layer)].add(self.nodes[layer_id])

        return layers


    def draw(self) -> NoReturn:
        # ????
        pyplot.figure(3, figsize=(12, 10))
        pyplot.gca().invert_yaxis()
#         nx.draw(self.graph,  # примитивная отрисовка по рассчитанным координатам, ничем не отличается от matplotlib
#                 nx.get_node_attributes(self.graph, Coord.COORD_ATTR), 
#                 with_labels=True, 
#                 node_size=10)
        pyplot.savefig(self.pic_path)

In [3]:
def add_to_dict(dct, k, v):
    if k in dct:
        dct[k].append(v)
    else:
        dct[k] = [v]

class BiEdges():
    """Класс для хранения прямых и инвертированных ребер"""

    def __init__(self):
        self.direct = {}
        self.invert = {}


class AcyclicGraph():
    """Класс ациклического графа"""
    
    def __init__(self, file_name, W=None):
        self.G = nx.read_graphml(file_name)
        self.W = W
        self.bi_edges = self.build_bi_edges()
        self.pi = {node:float('inf') for node in self.G.nodes()}

    def build_bi_edges(self):       
        """Строит 2 списка ребер (прямых и инвертированных)"""
        res = BiEdges()
        for edge in self.G.edges():
            add_to_dict(res.direct, edge[0], edge[1])
            add_to_dict(res.invert, edge[1], edge[0])
        return res

    def __compare_sets(self, a, b):
        """Сравнение множеств"""
        return tuple(a) < tuple(b)

    def topological_sort(self):
        """Топологическая сортировка вершин"""
        v_residual = set(self.G.nodes())
        sorted_pi = sorted([self.pi[node] for node in self.G.nodes()], reverse=True)
        for i in range(len(self.G.nodes())):
            vertex_to_update, min_set = None, sorted_pi
            for node in v_residual:
                if node in self.bi_edges.invert:
                    current_set = sorted(map(lambda x: self.pi[x], self.bi_edges.invert[node]), reverse=True)
                else:
                    current_set = set()
                if self.__compare_sets(current_set, min_set):
                    min_set = current_set
                    vertex_to_update = node
            self.pi[vertex_to_update] = i
            v_residual.remove(vertex_to_update)
    
    def greedy_layering(self):
        """Жадное присовение вершинам номер слоя"""
        v_residual = set(self.G.nodes())
        u = set()
        layers = [set()]
        while len(u) < len(self.G.nodes()):
            max_node = None
            max_pi = -1
            for node in v_residual:
                if node in self.bi_edges.direct:
                    is_subset = set(self.bi_edges.direct[node]) <= u
                else:
                    is_subset = True
                if is_subset and self.pi[node] > max_pi:
                    max_node = node
                    max_pi = self.pi[node]
            if max_node in self.bi_edges.direct:
                is_intersected = len(set(self.bi_edges.direct[max_node]).intersection(layers[-1])) > 0
            else:
                is_intersected = False
            if len(layers[-1]) >= self.W or is_intersected:
                layers.append(set())
            layers[-1].add(max_node)
            v_residual.remove(max_node)
            u.add(max_node)
        return layers

    def minimize_dummy_vertexes(self):
        """Минимизация dummy-вершин"""
        nodes = list(self.G.nodes())
        y_bounds = [(0, None) for _ in range(len(nodes))]
        node_to_id = {nodes[i]:i for i in range(len(nodes))}
        c = [0 for _ in range(len(nodes))]
        A = []
        for edge in self.G.edges():
            cur_bound = [0 for _ in range(len(nodes))]
            cur_bound[node_to_id[edge[0]]] = -1
            cur_bound[node_to_id[edge[1]]] = 1
            A.append(cur_bound)
            c[node_to_id[edge[0]]] += 1
            c[node_to_id[edge[1]]] -= 1

        b = [-1 for _ in range(len(A))]
        node_layers = linprog(c, A_ub=A, b_ub=b, bounds=y_bounds, method='revised simplex')['x']
        max_layer = int(max(node_layers))
        layers = [set() for i in range(max_layer+1)]
        for i in range(len(node_layers)):
            layers[int(node_layers[i])].add(nodes[i])
        return layers

    def __add_dummy_vertexes(self, layers, node_to_layer):
        """Добавление dummy-вершин в граф"""
        colors = {node:'#1f78b4' for node in self.G.nodes()}
        dummy_name = 'dummy_{0}'
        i = 0
        edges = list(self.G.edges())
        for edge in edges:
            if node_to_layer[edge[0]] - node_to_layer[edge[1]] > 1:
                prev_node = edge[0]
                self.G.remove_edge(*edge)
                for layer_num in range(node_to_layer[edge[0]]-1, node_to_layer[edge[1]], -1):
                    cur_dummy_name = dummy_name.format(i)
                    i += 1
                    layers[layer_num].add(cur_dummy_name)
                    self.G.add_node(cur_dummy_name, color='#FF0000')
                    self.G.add_edge(prev_node, cur_dummy_name)
                    prev_node = cur_dummy_name
                    colors[cur_dummy_name] = '#FF0000'
                self.G.add_edge(prev_node, edge[1])
        nx.set_node_attributes(self.G, colors, 'color')
        self.build_bi_edges()
        return layers

    def __insert_into_sorted(self, lst, elem):
        for i in range(len(lst)):
            if elem < lst[i]:
                return lst[:i] + [elem] + lst[i:]
        return lst + [elem]

    def minimize_intersections(self, layers):
        """Минимизация числа пересечений.
        Координата вершины - сред. арифм координат ее потомков"""
        node_to_layer = {}
        for i in range(len(layers)):
            for node in layers[i]:
                node_to_layer[node] = i
        layers = self.__add_dummy_vertexes(layers, node_to_layer)
        layer_0 = list(layers[0])
        coords = {layer_0[i]:(i, 0) for i in range(len(layer_0))}
        for i in range(1, len(layers)):
            layer = list(layers[i])
            point_dct = {}
            for j in range(len(layer)):
                x_coord = j
                if layer[j] in self.bi_edges.direct and len(self.bi_edges.direct[layer[j]]) > 0:
                    x_coord = sum(map(lambda x: coords[x][0], self.bi_edges.direct[layer[j]]))\
                     // len(self.bi_edges.direct[layer[j]])
                add_to_dict(point_dct, x_coord, layer[j])
            x_coords = sorted(point_dct.keys())
            point_num = 0
            while point_num < len(x_coords):
                x_coord = x_coords[point_num]
                if len(point_dct[x_coord]) > 1:
                    if x_coord + 1 in point_dct:
                        point_dct[x_coord + 1] = point_dct[x_coord][1:] + point_dct[x_coord + 1]
                    else:
                        point_dct[x_coord + 1] = point_dct[x_coord][1:]
                        x_coords = self.__insert_into_sorted(x_coords, x_coord + 1)
                point_dct[x_coord] = point_dct[x_coord][0]
                coords[point_dct[x_coord]] = (x_coord, i)
                point_num += 1
        return coords 

    def show(self):
        """"Изображение графа"""
        if self.W is not None:
            xticks_max = self.W
            self.topological_sort()
            layers = self.greedy_layering()
        else:
            layers = self.minimize_dummy_vertexes()
            xticks_max = len(max(layers, key=lambda x: len(x)))
        coords = self.minimize_intersections(layers) #минимизация числа пересечений с использованием эвристики

        nx.set_node_attributes(self.G, coords, 'pos')
        color_dct = nx.get_node_attributes(self.G, 'color')
        fig, ax = plt.subplots(figsize=(10,8))
        nx.draw_networkx(
            self.G,
            nx.get_node_attributes(self.G, 'pos'),
            with_labels=False,
            node_size=100,
            node_color=[color_dct[node] for node in self.G.nodes()],
            ax=ax
        )
        ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
        ax.set_xticks(range(0, xticks_max+1, 2))
        plt.grid('on')
        plt.show()

In [4]:
dag9nodes = """<?xml version="1.0" encoding="UTF-8"?>
<graphml xmlns="http://graphml.graphdrawing.org/xmlns"  
  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
  xsi:schemaLocation="http://graphml.graphdrawing.org/xmlns
  http://graphml.graphdrawing.org/xmlns/1.0/graphml.xsd">
  <graph id="flow" edgedefault="directed">
    <node id="n0"/>
    <node id="n1"/>
    <node id="n2"/>
    <node id="n3"/>
    <node id="n4"/>
    <node id="n5"/>
    <node id="n6"/>
    <node id="n7"/>
    <node id="n8"/>
    <edge source="n0" target="n1"/>
    <edge source="n1" target="n2"/>
    <edge source="n1" target="n4"/>
    <edge source="n2" target="n5"/>
    <edge source="n3" target="n4"/>
    <edge source="n4" target="n7"/>
    <edge source="n5" target="n6"/>
    <edge source="n5" target="n7"/>
    <edge source="n7" target="n8"/>
  </graph>
</graphml>"""

path = 'dag9nodes.xml'

with open(path, 'w') as f:
    f.write(dag9nodes)

In [5]:
drawer = AcyclicGraphDrawer('dag9nodes.xml')
drawer.compute_layers()

<class 'networkx.classes.digraph.DiGraph'>


[{'n8'}, {'n6', 'n7'}, {'n4', 'n5'}, {'n2', 'n3'}, {'n1'}, {'n0'}]

In [6]:
fuck_drawer = AcyclicGraph('dag9nodes.xml')
fuck_drawer.minimize_dummy_vertexes()

[{'n8'}, {'n6', 'n7'}, {'n4', 'n5'}, {'n2', 'n3'}, {'n1'}, {'n0'}]

In [7]:
dag13nodes = """<?xml version="1.0" encoding="UTF-8"?>
<graphml xmlns="http://graphml.graphdrawing.org/xmlns"
  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
  xsi:schemaLocation="http://graphml.graphdrawing.org/xmlns
  http://graphml.graphdrawing.org/xmlns/1.0/graphml.xsd">
  <graph id="tree" edgedefault="directed">
    <node id="n1"/>
    <node id="n2"/>
    <node id="n3"/>
    <node id="n4"/>
    <node id="n5"/>
    <node id="n6"/>
    <node id="n7"/>
    <node id="n8"/>
    <node id="n9"/>
    <node id="n10"/>
    <node id="n11"/>
    <node id="n12"/>
    <node id="n13"/>
    <edge source="n1" target="n3"/>
    <edge source="n1" target="n4"/>
    <edge source="n3" target="n5"/>
    <edge source="n3" target="n6"/>
    <edge source="n3" target="n8"/>
    <edge source="n3" target="n9"/>
    <edge source="n4" target="n7"/>
    <edge source="n4" target="n8"/>
    <edge source="n4" target="n10"/>
    <edge source="n5" target="n11"/>
    <edge source="n7" target="n9"/>
    <edge source="n8" target="n10"/>
    <edge source="n9" target="n12"/>
    <edge source="n10" target="n11"/>
    <edge source="n10" target="n12"/>
    <edge source="n11" target="n13"/>
    <edge source="n12" target="n13"/>
  </graph>
</graphml>"""

path = 'dag13nodes.xml'

with open(path, 'w') as f:
    f.write(dag13nodes)

In [8]:
drawer = AcyclicGraphDrawer('dag13nodes.xml')
drawer.compute_layers()

<class 'networkx.classes.digraph.DiGraph'>


[{'n13', 'n2'},
 {'n11', 'n12'},
 {'n10', 'n5', 'n9'},
 {'n6', 'n7', 'n8'},
 {'n3', 'n4'},
 {'n1'}]

In [9]:
fuck_drawer = AcyclicGraph('dag13nodes.xml')
fuck_drawer.minimize_dummy_vertexes()

[{'n13', 'n2'},
 {'n11', 'n12'},
 {'n10', 'n5', 'n9'},
 {'n6', 'n7', 'n8'},
 {'n3', 'n4'},
 {'n1'}]

In [10]:
dag44nodes = """<?xml version="1.0" encoding="UTF-8"?>
<graphml xmlns="http://graphml.graphdrawing.org/xmlns"  
  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
  xsi:schemaLocation="http://graphml.graphdrawing.org/xmlns
  http://graphml.graphdrawing.org/xmlns/1.0/graphml.xsd">
  <graph id="flow" edgedefault="directed">
    <node id="n0"/>
    <node id="n1"/>
    <node id="n2"/>
    <node id="n3"/>
    <node id="n4"/>
    <node id="n5"/>
    <node id="n6"/>
    <node id="n7"/>
    <node id="n8"/>
    <node id="n9"/>
    <node id="n10"/>
    <node id="n11"/>
    <node id="n12"/>
    <node id="n13"/>
    <node id="n14"/>
    <node id="n15"/>
    <node id="n16"/>
    <node id="n17"/>
    <node id="n18"/>
    <node id="n19"/>
    <node id="n20"/>
    <node id="n21"/>
    <node id="n22"/>
    <node id="n23"/>
    <node id="n24"/>
    <node id="n25"/>
    <node id="n26"/>
    <node id="n27"/>
    <node id="n28"/>
    <node id="n29"/>
    <node id="n30"/>
    <node id="n31"/>
    <node id="n32"/>
    <node id="n33"/>
    <node id="n34"/>
    <node id="n35"/>
    <node id="n36"/>
    <node id="n37"/>
    <node id="n38"/>
    <node id="n39"/>
    <node id="n40"/>
    <node id="n41"/>
    <node id="n42"/>
    <node id="n43"/>
    <edge source="n0" target="n6"/>
    <edge source="n1" target="n31"/>
    <edge source="n2" target="n7"/>
    <edge source="n2" target="n17"/>
    <edge source="n2" target="n22"/>
    <edge source="n3" target="n14"/>
    <edge source="n4" target="n12"/>
    <edge source="n5" target="n6"/>
    <edge source="n5" target="n10"/>
    <edge source="n6" target="n8"/>
    <edge source="n6" target="n14"/>
    <edge source="n7" target="n16"/>
    <edge source="n7" target="n19"/>
    <edge source="n8" target="n16"/>
    <edge source="n8" target="n22"/>
    <edge source="n9" target="n11"/>
    <edge source="n10" target="n11"/>
    <edge source="n10" target="n16"/>
    <edge source="n11" target="n12"/>
    <edge source="n12" target="n14"/>
    <edge source="n12" target="n17"/>
    <edge source="n13" target="n19"/>
    <edge source="n14" target="n20"/>
    <edge source="n14" target="n25"/>
    <edge source="n15" target="n21"/>
    <edge source="n15" target="n31"/>
    <edge source="n16" target="n18"/>
    <edge source="n16" target="n28"/>
    <edge source="n17" target="n18"/>
    <edge source="n17" target="n19"/>
    <edge source="n17" target="n28"/>
    <edge source="n18" target="n26"/>
    <edge source="n19" target="n23"/>
    <edge source="n20" target="n23"/>
    <edge source="n21" target="n27"/>
    <edge source="n22" target="n25"/>
    <edge source="n23" target="n24"/>
    <edge source="n23" target="n26"/>
    <edge source="n24" target="n30"/>
    <edge source="n25" target="n33"/>
    <edge source="n25" target="n40"/>
    <edge source="n26" target="n27"/>
    <edge source="n26" target="n31"/>
    <edge source="n27" target="n29"/>
    <edge source="n27" target="n40"/>
    <edge source="n28" target="n33"/>
    <edge source="n29" target="n30"/>
    <edge source="n29" target="n32"/>
    <edge source="n30" target="n33"/>
    <edge source="n31" target="n33"/>
    <edge source="n32" target="n34"/>
    <edge source="n33" target="n35"/>
    <edge source="n33" target="n36"/>
    <edge source="n33" target="n41"/>
    <edge source="n34" target="n36"/>
    <edge source="n36" target="n37"/>
    <edge source="n37" target="n38"/>
    <edge source="n37" target="n42"/>
    <edge source="n38" target="n39"/>
    <edge source="n39" target="n43"/>
    <edge source="n40" target="n42"/>
    <edge source="n41" target="n42"/>
  </graph>
</graphml>"""

path = 'dag44nodes.xml'

with open(path, 'w') as f:
    f.write(dag44nodes)

In [11]:
drawer = AcyclicGraphDrawer('dag44nodes.xml')
drawer.compute_layers()

<class 'networkx.classes.digraph.DiGraph'>


[{'n43'},
 {'n39'},
 {'n38', 'n42'},
 {'n37', 'n41'},
 {'n35', 'n36'},
 {'n33', 'n34'},
 {'n30', 'n32'},
 {'n24', 'n29', 'n40'},
 {'n25', 'n27', 'n31'},
 {'n1', 'n21', 'n26'},
 {'n15', 'n18', 'n23', 'n28'},
 {'n16', 'n19', 'n20', 'n22'},
 {'n13', 'n14', 'n17', 'n7', 'n8'},
 {'n12', 'n2', 'n3', 'n6'},
 {'n0', 'n11', 'n4'},
 {'n10', 'n9'},
 {'n5'}]